In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from community import community_louvain
import pandas as pd

In [2]:
def measures(G):
    def momment_of_degree_distribution(G,m):
        M = 0
        N = len(G)
        for i in G.nodes:
            M = M + G.degree(i)**m
        M = M/N
        return M

    N = len(G)
    M = G.number_of_edges()
    vk = dict(G.degree())
    vk = list(vk.values())
    vk = np.array(vk)
    k1 = momment_of_degree_distribution(G,1)
    k2 = momment_of_degree_distribution(G,2)
    variance = momment_of_degree_distribution(G,2) - momment_of_degree_distribution(G,1)**2
    av_cl = nx.average_clustering(G)
    l = nx.average_shortest_path_length(G)
    r=nx.degree_assortativity_coefficient(G)

    CLC = dict(nx.closeness_centrality(G))
    CLC = list(CLC.values())
    av_clc = np.mean(CLC) # Average closeness centrality

    B = dict(nx.betweenness_centrality(G))
    B = list(B.values())
    av_B = np.mean(B) #Average betweenness centrality

    EC = dict(nx.eigenvector_centrality(G, max_iter = 1000))
    EC = list(EC.values())
    av_EC = np.mean(EC) #Average eigenvector centrality

    PR = dict(nx.pagerank(G, alpha=0.85))
    PR = list(PR.values())
    av_pr = np.mean(PR) #Average PageRank

    def modularity(G, c):
        A = nx.adjacency_matrix(G)
        N = len(G)
        M = G.number_of_edges()
        Q = 0
        for i in np.arange(0,N):
            ki = len(list(G.neighbors(i)))
            for j in np.arange(0,N):
                if(c[i]==c[j]):
                    kj = len(list(G.neighbors(j)))
                    Q = Q + A[i,j]-(ki*kj)/(2*M)
        Q = Q/(2*M)
        return Q
    communities =list(community_louvain.best_partition(G).values())
    mod_louvrain = modularity(G,communities) # modularidade usando método de Louvrain

    return  k1,k2,variance,av_cl,l,r, av_clc, av_B, av_EC, av_pr, mod_louvrain

In [3]:
cl = ['ER','WS','BA']
#ER networks
X = []
y = []
n_nets = 30
av_degree = k1
p = av_degree/(N-1)
for i in range(0,n_nets):
    GER = nx.gnp_random_graph(N, p, seed=None, directed=False)
    Gcc = sorted(nx.connected_components(GER), key=len, reverse=True)
    GER = GER.subgraph(Gcc[0])
    GER = nx.convert_node_labels_to_integers(GER, first_label=0)
    k1,k2,variance,av_cl,l,r, av_clc, av_B, av_EC, av_pr, mod_louvrain = measures(GER)
    x = [k1,k2,variance,av_cl,l,r, av_clc, av_B, av_EC, av_pr, mod_louvrain]
    X.append(x)
    y.append(0.0)

#WS networks
k = int(av_degree)
p = 0.1 #probability of rewiring
for i in range(0,n_nets):
    GWS = nx.watts_strogatz_graph(N, k, p, seed=None)
    Gcc = sorted(nx.connected_components(GWS), key=len, reverse=True)
    GWS = GWS.subgraph(Gcc[0])
    GWS = nx.convert_node_labels_to_integers(GWS, first_label=0)
    k1,k2,variance,av_cl,l,r, av_clc, av_B, av_EC, av_pr, mod_louvrain = measures(GWS)
    x = [k1,k2,variance,av_cl,l,r, av_clc, av_B, av_EC, av_pr, mod_louvrain]
    X.append(x)
    y.append(1.0)

# BA networks
m = int(av_degree/2)
for i in range(0,n_nets):
    GBA = nx.barabasi_albert_graph(N, m)    
    Gcc = sorted(nx.connected_components(GBA), key=len, reverse=True)
    GBA = GBA.subgraph(Gcc[0])
    GBA = nx.convert_node_labels_to_integers(GBA, first_label=0)
    k1,k2,variance,av_cl,l,r, av_clc, av_B, av_EC, av_pr, mod_louvrain = measures(GWS)
    x = [k1,k2,variance,av_cl,l,r, av_clc, av_B, av_EC, av_pr, mod_louvrain]
    X.append(x)
    y.append(2.0)

NameError: name 'k1' is not defined

In [19]:
X = np.array(X)
y = np.array(y)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [20]:
from sklearn.neighbors import KNeighborsClassifier
k = 5
model = KNeighborsClassifier(n_neighbors=k, metric = 'euclidean')
model.fit(X,y)

from sklearn.ensemble import RandomForestClassifier

model2 = RandomForestClassifier(random_state = 37)
model2.fit(X,y)

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [21]:
G= nx.read_edgelist('/content/drive/Shareddrives/Redes_complexas/lesmis.txt', nodetype=int, data=(('weight',float),))
G = G.to_undirected()
G.remove_edges_from(nx.selfloop_edges(G))
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
G = nx.convert_node_labels_to_integers(G, first_label=0)
N = len(G)
M = G.number_of_edges()
print('Number of nodes:', N)
print('Number of edges:', M)

pos = nx.spring_layout(G)
nx.draw(G, pos, node_color="b", node_size=50, with_labels=False)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/Shareddrives/Redes_complexas/lesmis.txt'

In [22]:
k1,k2,variance,av_cl,l,r, av_clc, av_B, av_EC, av_pr, mod_louvrain = measures(G)
X_net = [k1,k2,variance,av_cl,l,r, av_clc, av_B, av_EC, av_pr, mod_louvrain]
print(X_net)

X_net = np.array(X_net)
X_net = X_net.reshape(1,len(X_net)) 
X_net = scaler.transform(X_net)
print('Xnet:', X_net.shape)

NameError: name 'G' is not defined

In [23]:
y_pred = model.predict(X_net) 
print('Classe (k-nearest neighbors):', cl[int(y_pred)])

NameError: name 'X_net' is not defined

In [24]:
y_pred = model2.predict(X_net) 
print('Classe (Random Forest):', cl[int(y_pred)])

NameError: name 'model2' is not defined

In [25]:
G= nx.read_edgelist('/content/drive/Shareddrives/Redes_complexas/lesmis.txt', nodetype=int, data=(('weight',float),))
G = G.to_undirected()
G.remove_edges_from(nx.selfloop_edges(G))
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
G = nx.convert_node_labels_to_integers(G, first_label=0)
N = len(G)
M = G.number_of_edges()
print('Number of nodes:', N)
print('Number of edges:', M)

pos = nx.spring_layout(G)
nx.draw(G, pos, node_color="b", node_size=50, with_labels=False)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/Shareddrives/Redes_complexas/lesmis.txt'

In [4]:
k1,k2,variance,av_cl,l,r, av_clc, av_B, av_EC, av_pr, mod_louvrain = measures(G)
X_net = [k1,k2,variance,av_cl,l,r, av_clc, av_B, av_EC, av_pr, mod_louvrain]


X_net = np.array(X_net)
X_net = X_net.reshape(1,len(X_net)) 
X_net = scaler.transform(X_net)
print('Xnet:', X_net.shape)

NameError: name 'G' is not defined

In [27]:
y_pred = model.predict(X_net) 
print('Classe (k-nearest neighbors):', cl[int(y_pred)])

NameError: name 'X_net' is not defined

In [28]:
y_pred = model2.predict(X_net) 
print('Classe (Random Forest):', cl[int(y_pred)])

NameError: name 'model2' is not defined

In [29]:
X_pd = pd.DataFrame(X, columns=['k1','k2','variance','av_cl','l','r', 'av_clc', 'av_B', 
                                'av_EC', 'av_pr', 'mod_louvrain'])
y_pd = pd.DataFrame(y)

plt.figure(figsize=(10,5))
rf_model = RandomForestClassifier(random_state = 37)
rf_model.fit(X, y)

random_forest = pd.Series(rf_model.feature_importances_, index=X_pd.columns)
random_forest.sort_values().plot(kind='barh')

NameError: name 'pd' is not defined